In [1]:
import tensorflow as tf

# 评估函数使用原理

In [2]:
m=tf.keras.metrics.Accuracy()
m.update_state([1,2,3,4],[0,2,3,4])

print('Final result: ',m.result().numpy())
# Final result: 0.75
m.update_state([1,2,3,4],[0,2,3,1])
print('Final result: ',m.result().numpy())


Final result:  0.75
Final result:  0.625


In [3]:
m=tf.keras.metrics.Accuracy()
m([1,2,3,4],[0,2,3,4])

print('Final result: ',m.result().numpy())
# Final result: 0.75
m([1,2,3,4],[0,2,3,1])
print('Final result: ',m.result().numpy())


Final result:  0.75
Final result:  0.625


In [4]:
m.reset_states()

In [5]:
m.update_state([1,2,3,4],[0,2,3,4])
print('Final result: ',m.result().numpy())

Final result:  0.75


# 自定义评估函数

In [9]:
class SparseCategoricalAccuracy_(tf.keras.metrics.Metric):
    def __init__(self, name='SparseCategoricalAccuracy', **kwargs):
        super(SparseCategoricalAccuracy_, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', dtype=tf.int32, initializer=tf.zeros_initializer())
        self.count = self.add_weight(name='count', dtype=tf.int32, initializer=tf.zeros_initializer())


    def update_state(self, y_true, y_pred,sample_weight=None):
        values = tf.cast(tf.equal(y_true, tf.argmax(y_pred, axis=-1, output_type=tf.int32)), tf.int32)
        self.total.assign_add(tf.shape(y_true)[0])
        self.count.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.count / self.total

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.total.assign(0)
        self.count.assign(0)

In [10]:
s = SparseCategoricalAccuracy_()
# s.reset_states()
s.update_state(tf.constant([2, 1]), tf.constant([[0.1, 0.9, 0.8], [0.05, 0.95, 0]]))
print('Final result: ', s.result().numpy())  # Final result: 0.5

Final result:  0.5


In [11]:
m = tf.keras.metrics.SparseCategoricalAccuracy()
m.update_state([2,1], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]])
print('Final result: ', m.result().numpy())  # Final result: 0.5

Final result:  0.5


In [12]:
class CatgoricalTruePositives(tf.keras.metrics.Metric):
    def __init__(self, name='categorical_true_positives', **kwargs):
        super(CatgoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred,axis=-1)
        values = tf.equal(tf.cast(y_true, 'int32'), tf.cast(y_pred, 'int32'))
        values = tf.cast(values, 'float32')
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, 'float32')
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.)


In [13]:
y_pred = tf.nn.softmax(tf.random.uniform((4,3)))

In [14]:
tf.argmax(y_pred,axis=-1)

<tf.Tensor: id=194, shape=(4,), dtype=int64, numpy=array([2, 2, 2, 2], dtype=int64)>

In [15]:
y_true = tf.constant([2,0,0,0])

In [16]:
m=CatgoricalTruePositives()
m.update_state(y_true,y_pred)

print('Final result: ',m.result().numpy())



Final result:  1.0


In [28]:
m.reset_states()